In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time


import pandas as pd
import collections
from functools import reduce

from math import erf, sqrt
from scipy.stats import norm
from statistics import mean

import random

import requests
from bs4 import BeautifulSoup

import time

pd.set_option('display.max_columns', None)


pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
expanded_standings = pd.read_csv('tundraB.csv', encoding= 'unicode_escape', index_col='Player')
current_day = expanded_standings['W'][1] + expanded_standings['L'][1] + expanded_standings['T'][1]
expanded_standings.columns = ['Rank', 'W', 'L', 'T', 'PTS', 'MPD', 'TMP', 'TCA', 'PCA',
       'UfPE', 'OE', 'sigma', 'Q%', 'TPA', 'CAA', 'PCAA', 'UfPA', 'DE', 'NUfP',
       'QPO', 'QPO+', 'QPD', 'QPD+', 'OPD', 'QvH', 'FW', 'FL', '3PT', 'MCW',
       'STR']
expanded_standings['mu'] = expanded_standings.TCA / current_day
expanded_standings['sigma2'] = expanded_standings.sigma
expanded_standings['p0'] = norm.cdf(x=0.5, loc=expanded_standings.mu, scale=expanded_standings.sigma).tolist()
expanded_standings['p1'] = (norm.cdf(1.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma']) - norm.cdf(0.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma'])).tolist()
expanded_standings['p2'] = (norm.cdf(2.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma']) - norm.cdf(1.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma'])).tolist()
expanded_standings['p3'] = (norm.cdf(3.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma']) - norm.cdf(2.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma'])).tolist()
expanded_standings['p4'] = (norm.cdf(4.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma']) - norm.cdf(3.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma'])).tolist()
expanded_standings['p5'] = (norm.cdf(5.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma']) - norm.cdf(4.5, loc=expanded_standings['mu'], scale=expanded_standings['sigma'])).tolist()
expanded_standings['p6'] = norm(expanded_standings['mu'], expanded_standings['sigma']).sf(5.5).tolist()

expanded_standings

,Rank,W,L,T,PTS,MPD,TMP,TCA,PCA,UfPE,OE,sigma,Q%,TPA,CAA,PCAA,UfPA,DE,NUfP,QPO,QPO+,QPD,QPD+,OPD,QvH,FW,FL,3PT,MCW,STR,mu,sigma2,p0,p1,p2,p3,p4,p5,p6
Player,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FloodMS,1,14,5,2,30,36,126,89,1.42,30,0.400,0.92,0.706,90,74,1.22,19,0.763,11,1.106,154,1.469,117,1.163,115,0,0,13,13,W2,4.238095,0.92,2.420776e-05,0.001435,0.027971,0.181766,0.400859,0.302857,0.085088
EnsonR,2,14,6,1,29,32,116,92,1.26,14,0.194,0.90,0.730,84,72,1.17,16,0.800,-2,0.925,84,1.530,126,0.994,119,0,0,10,9,W6,4.380952,0.90,8.083028e-06,0.000677,0.017627,0.145519,0.388786,0.340521,0.106863
ZimmermanP,3,13,6,2,28,29,123,96,1.28,11,0.172,0.85,0.762,94,75,1.25,22,0.728,-11,0.934,81,1.490,121,0.900,110,0,0,8,4,W2,4.571429,0.85,8.342559e-07,0.000150,0.007254,0.096338,0.362771,0.396165,0.137321
GarveyK2,4,13,6,2,28,17,97,76,1.28,27,0.307,0.90,0.603,80,65,1.23,22,0.722,5,0.910,105,1.325,95,1.028,99,1,0,8,6,W2,3.619048,0.90,2.645165e-04,0.009009,0.097589,0.340521,0.388786,0.145519,0.018311
IngramB,5,12,6,3,27,14,115,88,1.31,18,0.257,1.26,0.698,101,83,1.22,14,0.813,4,0.956,102,1.512,123,1.070,103,0,0,5,8,L3,4.190476,1.26,1.700549e-03,0.014668,0.073487,0.201992,0.305178,0.253644,0.149332
TelfordD,6,12,6,3,27,8,104,83,1.25,21,0.253,0.79,0.659,96,79,1.22,15,0.800,6,0.912,94,1.459,114,1.053,99,0,0,11,10,L1,3.952381,0.79,6.209412e-06,0.000948,0.032044,0.250449,0.472458,0.219040,0.025056
GiardinaB,7,11,8,2,24,14,112,82,1.37,29,0.363,0.97,0.651,98,80,1.23,18,0.775,11,1.013,132,1.426,110,1.138,111,0,0,10,9,W1,3.904762,0.97,2.239996e-04,0.006361,0.067194,0.264457,0.392039,0.219695,0.050029
StewartK3,8,11,6,4,24,6,127,90,1.41,20,0.357,1.55,0.789,121,83,1.46,37,0.543,-17,1.147,152,1.333,99,0.900,113,0,2,14,4,W1,4.285714,1.55,7.295060e-03,0.028854,0.088496,0.181463,0.248870,0.228329,0.216693
BassJ,9,10,8,3,23,-8,98,71,1.38,31,0.392,1.33,0.563,106,82,1.29,25,0.702,6,0.956,129,1.266,86,1.095,103,0,0,11,9,L2,3.380952,1.33,1.515064e-02,0.063494,0.175223,0.281794,0.264274,0.144515,0.055550


In [3]:
probs_table = [{0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9:0},
               {0: .1667, 1: .3333, 2: .3333, 3: .1667, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9:0},
               {0: 0, 1: .1111, 2: .2222, 3: .3333, 4: .2222, 5: .1111, 6: 0, 7: 0, 8: 0, 9:0},
               {0: 0, 1: 0, 2: .08333, 3: .16667, 4: .25, 5: .25, 6: .1667, 7: .08333, 8: 0, 9:0},
               {0: 0, 1: 0, 2: 0, 3: 0, 4: .1111, 5: .2222, 6: .3333, 7: .2222, 8: .1111, 9:0},
               {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: .1667, 7: .3333, 8: .3333, 9:.1667},
               {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9:1}]

pd.DataFrame(probs_table)

,0,1,2,3,4,5,6,7,8,9
0,1.0000,0.0000,0.00000,0.00000,0.0000,0.0000,0.0000,0.00000,0.0000,0.0000
1,0.1667,0.3333,0.33330,0.16670,0.0000,0.0000,0.0000,0.00000,0.0000,0.0000
2,0.0000,0.1111,0.22220,0.33330,0.2222,0.1111,0.0000,0.00000,0.0000,0.0000
3,0.0000,0.0000,0.08333,0.16667,0.2500,0.2500,0.1667,0.08333,0.0000,0.0000
4,0.0000,0.0000,0.00000,0.00000,0.1111,0.2222,0.3333,0.22220,0.1111,0.0000
5,1.0000,0.0000,0.00000,0.00000,0.0000,0.0000,0.1667,0.33330,0.3333,0.1667
6,0.0000,0.0000,0.00000,0.00000,0.0000,0.0000,0.0000,0.00000,0.0000,1.0000


In [4]:
def defensive_table(de):
    probs_table = [{0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9:0},
               {0: (2*.1667)*de, 1: (2*.3333)*de, 2: (2*.3333)*(1-de), 3: (2*.1667)*(1-de), 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9:0},
               {0: 0, 1: (2*.1111)*de, 2: (2*.2222)*de, 3: .3333, 4: (2*.2222)*(1-de), 5: (2*.1111)*(1-de), 6: 0, 7: 0, 8: 0, 9:0},
               {0: 0, 1: 0, 2: (2*.08333)*de, 3: (2*.1667)*de, 4: (2*.25)*de, 5: (2*.25)*(1-de), 6: (2*.1667)*(1-de), 7: (2*.08333)*(1-de), 8: 0, 9:0},
               {0: 0, 1: 0, 2: 0, 3: 0, 4: (2*.1111)*de, 5: (2*.2222)*de, 6: .3333, 7: (2*.2222)*(1-de), 8: (2*.1111)*(1-de), 9:0},
               {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: (2*.1667)*de, 7: (2*.3333)*(1-de), 8: (2*.3333)*(1-de), 9:(2*.1667)*(1-de)},
               {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9:1}]
    return pd.DataFrame(probs_table)

defensive_table(.9)

,0,1,2,3,4,5,6,7,8,9
0,1.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000
1,0.30006,0.59994,0.066660,0.03334,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000
2,0.00000,0.19998,0.399960,0.33330,0.04444,0.02222,0.00000,0.000000,0.00000,0.00000
3,0.00000,0.00000,0.149994,0.30006,0.45000,0.05000,0.03334,0.016666,0.00000,0.00000
4,0.00000,0.00000,0.000000,0.00000,0.19998,0.39996,0.33330,0.044440,0.02222,0.00000
5,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.30006,0.066660,0.06666,0.03334
6,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,1.00000


In [5]:
expanded_standings.loc[['BestJ']]['p0'][0]

0.0066812640116765505

In [6]:
def probs_list(player):
    p0 = expanded_standings.loc[[player]]['p0'][0]
    p1 = expanded_standings.loc[[player]]['p1'][0]
    p2 = expanded_standings.loc[[player]]['p2'][0]
    p3 = expanded_standings.loc[[player]]['p3'][0]
    p4 = expanded_standings.loc[[player]]['p4'][0]
    p5 = expanded_standings.loc[[player]]['p5'][0]
    p6 = expanded_standings.loc[[player]]['p6'][0]
    
    return [p0,p1,p2,p3,p4,p5,p6]

probs_list('Gozlan Z')

[0.00019589832548918225,
 0.006778250771523185,
 0.07811430201497978,
 0.30285655517771787,
 0.4008587469236586,
 0.1817660033486287,
 0.029430243438002664]

In [7]:
def heads_up_chances(player, opponent, sims):
    #get defensives
    de_x = expanded_standings.loc[[player]]['DE'][0]
    de_y = expanded_standings.loc[[opponent]]['DE'][0]
    
    probs_x = defensive_table(de_y).mul(probs_list(player), axis=0).sum(axis=0)
    probs_y = defensive_table(de_x).mul(probs_list(opponent), axis=0).sum(axis=0)
    
    x_sims = random.choices([0,1,2,3,4,5,6,7,8,9], weights=probs_x, k=sims)
    y_sims = random.choices([0,1,2,3,4,5,6,7,8,9], weights=probs_y, k=sims)
    
    win = 0
    lose = 0
    tie = 0
    
    p1_score = mean(x_sims)
    p2_score = mean(y_sims)
    
    for i in range(0,sims):
        if x_sims[i] > y_sims[i]:
            win = win+1
        if x_sims[i] < y_sims[i]:
            lose = lose+1
        if x_sims[i] == y_sims[i]:
            tie = tie+1
            
    return(win/sims, lose/sims, tie/sims, p1_score, p2_score)

In [8]:
heads_up_chances('Gozlan Z', 'DrugaE', 1000)

(0.514, 0.339, 0.147, 4.881, 4.27)

In [9]:
defensive_table(.5).mul(probs_list('Gozlan Z'), axis=0).sum(axis=0)

0    0.001326
1    0.010938
2    0.044853
3    0.077652
4    0.137607
5    0.173463
6    0.214393
7    0.174890
8    0.105118
9    0.059731
dtype: float64

## Schedule Finder

In [10]:
all_days = [str(i) for i in range(1,26)]
league = 'B_Tundra' #C_Tundra_Div_2
season = '92'
day = '1'

webpage = f'https://learnedleague.com/schedule.php?{season}&{day}&{league}'

webpage

'https://learnedleague.com/schedule.php?92&1&B_Tundra'

In [11]:
cagematch_html = requests.get(webpage, headers={'Accept-Encoding': 'identity'})
cagematch_html.encoding = cagematch_html.apparent_encoding
soup = BeautifulSoup(cagematch_html.text, features='html.parser')

quad_1 = [i.get_text() for i in soup.find_all("td", attrs={"class": " n"})]
quad_2 = [i.get_text() for i in soup.find_all("td", attrs={"class": " n r"})]
quad_3 = [i.get_text() for i in soup.find_all("td", attrs={"class": "odd-row n"})]
quad_4 = [i.get_text() for i in soup.find_all("td", attrs={"class": "odd-row n r"})]

In [12]:
all_days = [str(i) for i in range(1,26)]
league = 'B_Tundra' #C_Tundra_Div_2
season = '92'

schedule = pd.DataFrame(columns=['league', 'season', 'day', 'player1', 'player2'])

for i in all_days:
    time.sleep(1)
    day = i
    webpage = f'https://learnedleague.com/schedule.php?{season}&{day}&{league}'
    
    ll_html = requests.get(webpage, headers={'Accept-Encoding': 'identity'})
    ll_html.encoding = ll_html.apparent_encoding
    soup = BeautifulSoup(ll_html.text, features='html.parser')

    quad_1 = [i.get_text() for i in soup.find_all("td", attrs={"class": " n"})]
    quad_2 = [i.get_text() for i in soup.find_all("td", attrs={"class": " n r"})]
    quad_3 = [i.get_text() for i in soup.find_all("td", attrs={"class": "odd-row n"})]
    quad_4 = [i.get_text() for i in soup.find_all("td", attrs={"class": "odd-row n r"})]
    
    for i in range(0, len(quad_4)):
        schedule = schedule.append({'league': league, 'season': season, 'day': int(day), 'player1': quad_1[i], 'player2': quad_2[i]}, ignore_index=True)
        schedule = schedule.append({'league': league, 'season': season, 'day': int(day), 'player1': quad_3[i], 'player2': quad_4[i]}, ignore_index=True)
        
    if len(quad_4) != len(quad_1):
        schedule = schedule.append({'league': league, 'season': season, 'day': int(day), 'player1': quad_1[len(quad_1)-1], 'player2': quad_2[len(quad_1)-1]}, ignore_index=True)

In [13]:
start = time.time()
print("hello")

p1_win = []
p1_lose = []
p1_tie = []
p1_score = []
p2_score = []

for i in range (0, len(schedule)):
    x1, x2, x3, x4, x5 = heads_up_chances(schedule['player1'][i], schedule['player2'][i],1000)
    p1_win.append(x1)
    p1_lose.append(x2)
    p1_tie.append(x3)
    p1_score.append(x4)
    p2_score.append(x5)

schedule['p1_win'] = p1_win
schedule['p1_lose'] = p1_lose
schedule['p1_tie'] = p1_tie
schedule['p1_score'] = p1_score
schedule['p2_score'] = p2_score

end = time.time()
print(end - start)

schedule

hello
5.540227890014648


,league,season,day,player1,player2,p1_win,p1_lose,p1_tie,p1_score,p2_score
0,B_Tundra,92,1,BauerlinM,BassJ,0.516,0.364,0.120,5.360,4.741
1,B_Tundra,92,1,BestJ,IngramB,0.315,0.555,0.130,4.773,5.737
2,B_Tundra,92,1,ByrneS,GarveyK2,0.447,0.394,0.159,5.020,4.845
3,B_Tundra,92,1,DrugaE,Gozlan Z,0.332,0.538,0.130,4.271,4.942
4,B_Tundra,92,1,FloodMS,VelkerJ,0.527,0.341,0.132,5.857,5.019
...,...,...,...,...,...,...,...,...,...,...
370,B_Tundra,92,25,KrugK,GarveyK2,0.474,0.374,0.152,5.035,4.644
371,B_Tundra,92,25,MalteseJ,CullenS,0.424,0.427,0.149,5.132,5.118
372,B_Tundra,92,25,McNittS,ReederR,0.643,0.256,0.101,5.250,3.571
373,B_Tundra,92,25,RobinsonDrusoe,StewartK3,0.349,0.538,0.113,5.039,5.926


In [14]:
schedule[(schedule.player1 == 'Gozlan Z') | (schedule.player2 == 'Gozlan Z')]

,league,season,day,player1,player2,p1_win,p1_lose,p1_tie,p1_score,p2_score
3,B_Tundra,92,1,DrugaE,Gozlan Z,0.332,0.538,0.130,4.271,4.942
27,B_Tundra,92,2,TelfordD,Gozlan Z,0.489,0.351,0.160,5.497,4.987
31,B_Tundra,92,3,BestJ,Gozlan Z,0.411,0.459,0.130,4.991,5.181
51,B_Tundra,92,4,HovelandR,Gozlan Z,0.436,0.397,0.167,5.473,5.268
73,B_Tundra,92,5,WintnerT,Gozlan Z,0.477,0.390,0.133,5.329,5.101
83,B_Tundra,92,6,ShemonteeO,Gozlan Z,0.449,0.437,0.114,5.232,5.255
92,B_Tundra,92,7,CondronS2,Gozlan Z,0.385,0.471,0.144,4.555,4.921
116,B_Tundra,92,8,TemkinE,Gozlan Z,0.372,0.504,0.124,4.616,5.183
130,B_Tundra,92,9,StewartK3,Gozlan Z,0.494,0.392,0.114,5.923,5.571
141,B_Tundra,92,10,IngramB,Gozlan Z,0.563,0.290,0.147,5.817,4.914


In [15]:
remaining_sched = schedule[schedule['day'] > current_day]

In [16]:
wins1 = remaining_sched[['player1', 'p1_win', 'p1_lose', 'p1_score', 'p2_score']]
wins2 = remaining_sched[['player2', 'p1_win', 'p1_lose', 'p1_score', 'p2_score']]
wins3 = remaining_sched[['player1', 'p1_tie']]
wins4 = remaining_sched[['player2', 'p1_tie']]

wins1 = wins1.groupby('player1').sum()
wins2 = wins2.groupby('player2').sum()
wins3 = wins3.groupby('player1').sum()
wins4 = wins4.groupby('player2').sum()

wins1 = wins1.rename(columns={'player1':'player', 'p1_win': 'win', 'p1_lose': 'lose', 'p1_score': 'points_scored', 'p2_score': 'points_allowed'})
wins2 = wins2.rename(columns={'player2':'player', 'p1_lose': 'win', 'p1_win': 'lose', 'p2_score': 'points_scored', 'p1_score': 'points_allowed'})
wins3 = wins3.rename(columns={'player1':'player', 'p1_tie': 'tie'})
wins4 = wins4.rename(columns={'player2':'player', 'p1_tie': 'tie'})

wins_tot = pd.concat([wins1, wins2, wins3, wins4])

wins_tot['player'] = wins_tot.index
simmed_wins = wins_tot.groupby('player').sum()

simmed_wins = simmed_wins[['win', 'lose', 'tie', 'points_scored', 'points_allowed']]

In [17]:
new_standings = expanded_standings[['W', 'L', 'T', 'MPD', 'TMP', 'TPA', 'FL']]

new_standings['W'] = round(simmed_wins['win'] + expanded_standings['W'], 0)
new_standings['L'] = round(simmed_wins['lose'] + expanded_standings['L'], 0)
new_standings['T'] = round(simmed_wins['tie'] + expanded_standings['T'], 0)
new_standings['TMP'] = round(simmed_wins['points_scored'] + expanded_standings['TMP'], 0)
new_standings['TPA'] = round(simmed_wins['points_allowed'] + expanded_standings['TPA'], 0)
new_standings['MPD'] = round(new_standings['TMP'] - new_standings['TPA'], 0)
new_standings['FL'] = expanded_standings.FL * (-1)

for index, row in new_standings.iterrows():
    if row['W'] + row['L'] + row['T'] != 25:
        diff = 25 - round((row['W'] + row['L'] + row['T']),0)
        whatever = random.randint(1,3)
        if whatever == 1:
            new_standings.loc[index, 'W']= row['W'] + diff
        if whatever == 2:
            new_standings.loc[index, 'L']= row['L'] + diff
        if whatever == 3:
            new_standings.loc[index, 'T']= row['T'] + diff
        
new_standings['PTS'] = round((new_standings['W']*2) + (new_standings['T']*1) + (new_standings['FL']*(1)),0)

In [18]:
prediction = new_standings.sort_values(by=['PTS', 'MPD', 'TMP', 'FL', 'W'], ascending=False)
prediction['rank'] = range(1, len(prediction)+1)
prediction

,W,L,T,MPD,TMP,TPA,FL,PTS,rank
Player,,,,,,,,,
FloodMS,16.0,6.0,3.0,38.0,149.0,111.0,0,35.0,1
EnsonR,16.0,7.0,2.0,36.0,141.0,105.0,0,34.0,2
ZimmermanP,15.0,7.0,3.0,35.0,149.0,114.0,0,33.0,3
GarveyK2,15.0,7.0,3.0,17.0,117.0,100.0,0,33.0,4
IngramB,14.0,7.0,4.0,17.0,138.0,121.0,0,32.0,5
TelfordD,14.0,7.0,4.0,10.0,126.0,116.0,0,32.0,6
GiardinaB,13.0,9.0,3.0,17.0,134.0,117.0,0,29.0,7
StewartK3,13.0,8.0,4.0,8.0,151.0,143.0,-2,28.0,8
FrithP,12.0,10.0,3.0,21.0,141.0,120.0,0,27.0,9


In [19]:
all_predictions = pd.DataFrame(columns=['W', 'L', 'T', 'MPD', 'TMP', 'TPA', 'FL', 'rank'])

i = 0
number_sims = 1000 #ten seconds per

while i < number_sims: 
    current_standings = expanded_standings[['W', 'L', 'T', 'MPD', 'TMP', 'TPA', 'FL']]
    
    schedule_sim = schedule[schedule['day'] > current_day]
    schedule_sim = schedule_sim.reset_index()
    
    p1_win = []
    p1_lose = []
    p1_tie = []
    p1_score = []
    p2_score = []

    for j in range (0, len(schedule_sim)):
        x1, x2, x3, x4, x5 = heads_up_chances(schedule_sim['player1'][j], schedule_sim['player2'][j],1)
        p1_win.append(x1)
        p1_lose.append(x2)
        p1_tie.append(x3)
        p1_score.append(x4)
        p2_score.append(x5)

    schedule_sim['p1_win'] = p1_win
    schedule_sim['p1_lose'] = p1_lose
    schedule_sim['p1_tie'] = p1_tie
    schedule_sim['p1_score'] = p1_score
    schedule_sim['p2_score'] = p2_score

    remaining_sched = schedule_sim
    
    wins1 = remaining_sched[['player1', 'p1_win', 'p1_lose', 'p1_score', 'p2_score']]
    wins2 = remaining_sched[['player2', 'p1_win', 'p1_lose', 'p1_score', 'p2_score']]
    wins3 = remaining_sched[['player1', 'p1_tie']]
    wins4 = remaining_sched[['player2', 'p1_tie']]

    wins1 = wins1.groupby('player1').sum()
    wins2 = wins2.groupby('player2').sum()
    wins3 = wins3.groupby('player1').sum()
    wins4 = wins4.groupby('player2').sum()

    wins1 = wins1.rename(columns={'player1':'player', 'p1_win': 'win', 'p1_lose': 'lose', 'p1_score': 'points_scored', 'p2_score': 'points_allowed'})
    wins2 = wins2.rename(columns={'player2':'player', 'p1_lose': 'win', 'p1_win': 'lose', 'p2_score': 'points_scored', 'p1_score': 'points_allowed'})
    wins3 = wins3.rename(columns={'player1':'player', 'p1_tie': 'tie'})
    wins4 = wins4.rename(columns={'player2':'player', 'p1_tie': 'tie'})

    wins_tot = pd.concat([wins1, wins2, wins3, wins4])

    wins_tot['player'] = wins_tot.index
    simmed_wins = wins_tot.groupby('player').sum()

    simmed_wins = simmed_wins[['win', 'lose', 'tie', 'points_scored', 'points_allowed']]
    
    new_standings = expanded_standings[['W', 'L', 'T', 'MPD', 'TMP', 'TPA', 'FL']]

    new_standings['W'] = round(simmed_wins['win'] + expanded_standings['W'], 0)
    new_standings['L'] = round(simmed_wins['lose'] + expanded_standings['L'], 0)
    new_standings['T'] = round(simmed_wins['tie'] + expanded_standings['T'], 0)
    new_standings['TMP'] = round(simmed_wins['points_scored'] + expanded_standings['TMP'], 0)
    new_standings['TPA'] = round(simmed_wins['points_allowed'] + expanded_standings['TPA'], 0)
    new_standings['MPD'] = round(new_standings['TMP'] - new_standings['TPA'], 0)
    new_standings['FL'] = expanded_standings.FL * (-1)

    for index, row in new_standings.iterrows():
        if row['W'] + row['L'] + row['T'] != 25:
            diff = 25 - round((row['W'] + row['L'] + row['T']),0)
            whatever = random.randint(1,3)
            if whatever == 1:
                new_standings.at[index, 'W']= row['W'] + diff
            if whatever == 2:
                new_standings.at[index, 'L']= row['L'] + diff
            if whatever == 3:
                new_standings.at[index, 'T']= row['T'] + diff

    new_standings['PTS'] = round((new_standings['W']*2) + (new_standings['T']*1) + (new_standings['FL']*(1)),0)
    
    prediction = new_standings.sort_values(by=['PTS', 'MPD', 'TMP', 'FL', 'W'], ascending=False)
    prediction['rank'] = range(1, len(prediction)+1)
    
    all_predictions = pd.concat([all_predictions, prediction])
    
    i = i+1
    if i % 10 == 0:
        print(i)
    
all_predictions

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000


,W,L,T,MPD,TMP,TPA,FL,rank,PTS
EnsonR,17.0,7.0,1.0,35.0,140.0,105.0,0,1,35.0
IngramB,15.0,6.0,4.0,24.0,144.0,120.0,0,2,34.0
TelfordD,15.0,6.0,4.0,13.0,123.0,110.0,0,3,34.0
FloodMS,15.0,7.0,3.0,28.0,145.0,117.0,0,4,33.0
ZimmermanP,15.0,8.0,2.0,33.0,149.0,116.0,0,5,32.0
...,...,...,...,...,...,...,...,...,...
DrugaE,8.0,13.0,4.0,-14.0,107.0,121.0,0,26,20.0
CondronS2,7.0,15.0,3.0,-23.0,101.0,124.0,0,27,17.0
FrascoS,8.0,16.0,1.0,-21.0,119.0,140.0,-1,28,16.0
MalteseJ,4.0,15.0,6.0,-39.0,106.0,145.0,0,29,14.0


In [20]:
all_predictions

,W,L,T,MPD,TMP,TPA,FL,rank,PTS
EnsonR,17.0,7.0,1.0,35.0,140.0,105.0,0,1,35.0
IngramB,15.0,6.0,4.0,24.0,144.0,120.0,0,2,34.0
TelfordD,15.0,6.0,4.0,13.0,123.0,110.0,0,3,34.0
FloodMS,15.0,7.0,3.0,28.0,145.0,117.0,0,4,33.0
ZimmermanP,15.0,8.0,2.0,33.0,149.0,116.0,0,5,32.0
...,...,...,...,...,...,...,...,...,...
DrugaE,8.0,13.0,4.0,-14.0,107.0,121.0,0,26,20.0
CondronS2,7.0,15.0,3.0,-23.0,101.0,124.0,0,27,17.0
FrascoS,8.0,16.0,1.0,-21.0,119.0,140.0,-1,28,16.0
MalteseJ,4.0,15.0,6.0,-39.0,106.0,145.0,0,29,14.0


In [21]:
all_predictions['player'] = all_predictions.index
outcome_dist = pd.pivot_table(all_predictions, values = ['rank'], index = ['player'], columns=all_predictions['rank'].values, aggfunc='count', fill_value=0)

In [22]:
outcome_dist['promoted'] = (outcome_dist['rank'][1] + outcome_dist['rank'][2] + outcome_dist['rank'][3]) / number_sims

floor = len(expanded_standings)
outcome_dist['relegated'] = (outcome_dist['rank'][floor] + outcome_dist['rank'][floor-1] 
                            + outcome_dist['rank'][floor-2] + outcome_dist['rank'][floor-3]
                            + outcome_dist['rank'][floor-4] + outcome_dist['rank'][floor-5]) / number_sims

outcome_dist.sort_values(by=['promoted', 'relegated'], ascending=[False, True]).to_csv('tundraB_1k_19b.csv')